# Proyecto 1: búsqueda de árbol
## Robot en un almacén con obstáculos

Un robot debe moverse en un almacén representado por una cuadrícula donde algunas celdas están bloqueadas. El robot comienza en una celda inicial y debe llegar a una meta determinada. Se pueden permitir movimientos en cuatro direcciones $\left(N, S, O, E\right)$.

![](maze.png)

Este problema puede formularse como sigue:

- **Estados:** cualquier ubicación que se encuentre el robot en una celda desbloqueada.
- **Estado inicial:** ubicación del robot sobre una celda desbloqueada.
- **Función sucesora:** generar estados legales al aplicar los movimientos en las direcciones $\left(N, S, O, E\right)$.
- **Condición meta:** que el robot se encuentre sobre la celda objetivo desbloqueda.

Antes de comenzar, importamos el módulo **SimpleSearch**, el cual contiene las clases para instanciar nodos (*node*) y realizar las búsquedas (*BlindSearch*).

Un nodo del módulo **SimpleSearch** tiene los siguientes componentes:

- **state**: el estado del sistema en este nodo.
- **parent**: nodo que lo generó (predeterminado *None*). 
- **depth**: Profundidad del árbol del nodo (valor predeterminado 0).
- **cost**: costo del nodo (es decir, uno por movimiento, predeterminado 0).
- **delta**: valor heurístico del nodo (predeterminado 0).
- **op**: cadena que contiene la descripción de la operación del generador (predeterminado "").

In [22]:
import sys,os
import numpy as np

# Obtenemos la ruta absoluta de la ruta relativa "../src"
# i.e. "../src" -> "/home/juandto/FIE/Inteligencia_artificial/searchs_ia/src" 
baile_path = os.path.abspath(os.path.join("..", "src"))

# sys.path contiene una lista de directorios donde el intérprete buscará
# el módulo requerido
if not baile_path in sys.path:
  # Añadimos el directorio en caso de que no esté
  sys.path.append(baile_path)

import SimpleSearch as ss

Para la definición de un estado se utilizan tuplas con el fin de que el estado del nodo sea inmutable. A continuación, se definen los estados de inicio (`init_state`) y de meta (`fin_state`). Un 0 significa que es una celda libre; un 1 significa un muro; y el 2 es usado para representar al robot. Finalmente, estos estados se pasan como atributo para la creación de los nodos `start` y `final`.

In [23]:
init_state = (
    (2, 0, 0, 0, 0),
    (0, 1, 0, 1, 1),
    (0, 0, 0, 0, 0),
    (0, 1, 1, 1, 0),
    (0, 0, 0, 0, 0)
    )

start = ss.node(init_state, op="start")

fin_state = (
    (0, 0, 0, 0, 0),
    (0, 1, 0, 1, 1),
    (0, 0, 0, 0, 0),
    (0, 1, 1, 1, 0),
    (0, 0, 0, 0, 2)
    )

final = ss.node(fin_state)

La función sucedora recibe un nodo y retorna una lista de sucedores. Si la posición del robot se encuentra en las coordenadas $\left(i,j\right)$, se debe verificar que pueda moverse en las direcciones $\left(N,S,O,E\right)$ tales que:

- $N = \left(i - 1, j\right)$
- $S = \left(i + 1, j\right)$
- $O = \left(i, j - 1\right)$
- $E = \left(i, j + 1\right)$

Se comprueba que la dirección a moverse no se salga del tablero o no sea una celda bloqueda (muro). Una vez hecho tales verificaciones, cada estado posible se añade a la lista de sucedores, y cuando termina, se retorna.

In [24]:
def successor(node):
  maze = node.state # maze = laberinto
  sucessors = []

  # Encontrar el robot (2) en el tablero. El resultado se almacena en index
  for index,value in np.ndenumerate(maze):
    if value == 2:
      break

  # Generar los estados válidos para los que podrá moverse el robot
  # (-1, 0) = N
  # (1, 0) = S
  # (0, -1) = O
  # (0, 1) = E
  x, y = index
  for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
    nx, ny = x + dx, y + dy
    if 0 <= nx < len(maze) and 0 <= ny < len(maze[0]) and maze[nx][ny] != 1:
      nmaze = [list(r[:]) for r in maze] # Se hace una copia del tablero original
      nmaze[x][y] = 0 # Se marca como vacía la posición antigua del robot
      nmaze[nx][ny] = 2 # Se marca como ocupada la nueva posición del robot
      nmaze = tuple([tuple(r) for r in nmaze]) # Hacer inmutable
      sucessors.append(ss.node(nmaze, depth=node.depth+1, op=f'robot moved from ({x}, {y}) to ({nx}, {ny})', parent=node))

  return sucessors

Para nuestro problema, la condición meta es dada de forma explicítica dado que conocemos la configuración del estado final. Como consecuencia, la función `goal` recibe dos nodos (`start` y `final`). Cuando `final` es dado implícitamente, la función `goal` solo recibe un argumento (i.g. un problema de ajedrez). Es por ello, que se utiliza la notación `*var` para indicar un número dinámico de argumentos.

In [25]:
def goal(*nodes):
  current = nodes[0]
  final = nodes[1]
  return current.state == final.state

A través de la clase `BlindSearch`, se crean dos objetos los cuales corresponden a las búsquedas ciegas que se aplicarán. En el constructor especificamos el nodo inicial, la función sucesora, el nodo final y la estrategía a utilizar (BFS o DFS).

In [26]:
bfs = ss.BlindSearch(start, successor, goal, goal_state=final, strategy="bfs")
dfs = ss.BlindSearch(start, successor, goal, goal_state=final, strategy="dfs")

Primero ejecutamos la búsqueda BFS. Para ello, se ejecuta el método `find`. Si la búsqueda tiene éxito, entonces regresa un nodo en el estado final, de lo contrario, se imprime que la solución no fue encontrada.

In [27]:
resb = bfs.find() # Realiza BFS

finished in 19 iterations
Se acabaron los estados: 0


Visualizamos el estado retornado por la búsqueda BFS, el cual debe corresponder al estado meta definido en un inicio.

In [28]:
print(resb.state)

((0, 0, 0, 0, 0), (0, 1, 0, 1, 1), (0, 0, 0, 0, 0), (0, 1, 1, 1, 0), (0, 0, 0, 0, 2))


Obtenemos la ruta de solución a través del método `getPath`.

In [29]:
seqb = resb.getPath()

Se imprime el número de pasos que le tomó a la búsqueda llegar hasta la solución, y también, la secuencia de la ruta.

In [ ]:
print("La secuencia de solución tiene %s pasos" %len(seqb))
seqb # Solución

La secuencia de solución tiene 9 pasos


[(((2, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'start',
  0),
 (((0, 2, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (0, 0) to (0, 1)',
  1),
 (((0, 0, 2, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (0, 1) to (0, 2)',
  2),
 (((0, 0, 0, 0, 0),
   (0, 1, 2, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (0, 2) to (1, 2)',
  3),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 2, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (1, 2) to (2, 2)',
  4),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 2, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (2, 2) to (2, 3)',
  5),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 2),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (2, 3) to (2, 4)',
  6),
 (((0,

Ahora se ejecuta la búsqueda DFS.

In [31]:
resd = dfs.find() # Realiza DFS

finished in 8 iterations
Se acabaron los estados: 3


Se obtiene la ruta de solución.

In [32]:
seqd = resd.getPath()

Se imprime el total de pasos que llevan a la solución junto con la secuencia de la ruta.

In [33]:
print("La secuencia de solución tiene %s pasos" %len(seqd))
seqd

La secuencia de solución tiene 9 pasos


[(((2, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'start',
  0),
 (((0, 0, 0, 0, 0),
   (2, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (0, 0) to (1, 0)',
  1),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (2, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (1, 0) to (2, 0)',
  2),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (2, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (2, 0) to (3, 0)',
  3),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (2, 0, 0, 0, 0)),
  'robot moved from (3, 0) to (4, 0)',
  4),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 2, 0, 0, 0)),
  'robot moved from (4, 0) to (4, 1)',
  5),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 2, 0, 0)),
  'robot moved from (4, 1) to (4, 2)',
  6),
 (((0,

### Búsqueda informada A*

Para una búsqueda informada, se requiere definir una heurística admisible. Esta se define como una función $h(n)$ (para nuestro caso, es $manhattan(nodes)$), la cual recibe uno o varios nodos y regresa el coste mínimo entre el nodo inicial y el nodo objetivo. Este coste debe ser menor que el coste ideal de solución $g(n)$.

Nuestra heurística se basa en la idea de calcular la distancia manhattan entre el nodo inicial (`start`) y meta (`final`), ignorando las celdas obstáculo.

In [34]:
def manhattan(*nodes):
  current_s = nodes[0].state
  goal_s = nodes[1].state

  
  # Encontrar el robot (2) en el tablero. El resultado se almacena en index_c
  for index_c,value in np.ndenumerate(current_s):
    if value == 2:
      break

  # Encontrar la posición final del robot (2) en el tablero. El resultado se almacena en index_g
  for index_g,value in np.ndenumerate(goal_s):
    if value == 2:
      break 

  # Calcular los pasos horizontales y verticales para la meta
  dis = (abs(index_g[0] - index_c[0]), abs(index_g[1] - index_c[1]))  
  
  # Retornar los pasos totales
  return dis[0] + dis[1]

In [35]:
manhattan(start, final)

8

Se crea una instancia para la búsqueda A*. Se pasan como parámetros los nodos inicial y meta, función sucesora, función meta, la estrategía y la heurística.

In [36]:
astars = ss.BlindSearch(start, successor, goal, goal_state=final, strategy="a*", heuristic=manhattan)

Se ejecuta la búsqueda.

In [37]:
resa = astars.find()

finished in 19 iterations
Se acabaron los estados: 0


Se obtiene la ruta de solución.

In [38]:
seqa = resa.getPath()

Se muestran el total de pasos para llegar a la solución y la secuencia de la ruta.

In [39]:
print("La secuencia de solución tiene %s pasos" %len(seqa))
seqa

La secuencia de solución tiene 9 pasos


[(((2, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'start',
  0),
 (((0, 0, 0, 0, 0),
   (2, 1, 0, 1, 1),
   (0, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (0, 0) to (1, 0)',
  1),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (2, 0, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (1, 0) to (2, 0)',
  2),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 2, 0, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (2, 0) to (2, 1)',
  3),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 2, 0, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (2, 1) to (2, 2)',
  4),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 2, 0),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (2, 2) to (2, 3)',
  5),
 (((0, 0, 0, 0, 0),
   (0, 1, 0, 1, 1),
   (0, 0, 0, 0, 2),
   (0, 1, 1, 1, 0),
   (0, 0, 0, 0, 0)),
  'robot moved from (2, 3) to (2, 4)',
  6),
 (((0,

Finalmente, se obtiene el tiempo de ejecución de cada búsqueda para la comparación de resultados.

In [40]:
%time bfs.find() # Tiempo de ejecución de BFS

finished in 19 iterations
Se acabaron los estados: 0
CPU times: user 1.12 ms, sys: 0 ns, total: 1.12 ms
Wall time: 2.75 ms


In [41]:
%time dfs.find() # Tiempo de ejecución de DFS

finished in 8 iterations
Se acabaron los estados: 3
CPU times: user 568 μs, sys: 0 ns, total: 568 μs
Wall time: 639 μs


In [42]:
%time astars.find() # Tiempo de ejecución de A*

finished in 19 iterations
Se acabaron los estados: 0
CPU times: user 1.99 ms, sys: 1.03 ms, total: 3.02 ms
Wall time: 13.5 ms


### Conclusión

|Estrategía de búsqueda|No. de iteraciones|Tiempo de ejecución|No. de pasos|
|:-------------:|--------------:|-------------:|--------------:|
|BFS|19|2.75 ms|9|
|DFS|8|639 μs|9|
|$A^*$|19|13.5 ms|9|

Referente a los resultados observados en la tabla, podemos determinar que la búsqueda DFS fue la más óptima en términos de tiempo e iteraciones. Sin embargo, esto no implica que este método sea el mejor en todos los casos. En general, la búsqueda DFS encuentra una solución bastante rápido pero no significa que sea la más óptima. Por otro lado, BFS tarda mucho más en encontrar una solución (en términos de espacio y tiempo), sin embargo, la solución que encuentra es la más eficiente.